# 实现softmax_loss_function部分
### 基于tensorflow1.4 Seq2seq的实现

In [1]:
import helpers
import tensorflow as tf
from tensorflow.contrib import seq2seq,rnn

tf.__version__

'1.4.0'

In [2]:
tf.reset_default_graph()
sess = tf.InteractiveSession()

In [3]:
PAD = 0
EOS = 1


vocab_size = 10
input_embedding_size = 20
encoder_hidden_units = 25

decoder_hidden_units = encoder_hidden_units

import helpers as data_helpers
batch_size = 50

# 一个generator，每次产生一个minibatch的随机样本

batches = data_helpers.random_sequences(length_from=3, length_to=8,
                                   vocab_lower=2, vocab_upper=10,
                                   batch_size=batch_size)

print('产生%d个长度不一（最短3，最长8）的sequences, 其中前十个是:' % batch_size)
for seq in next(batches)[:min(batch_size, 10)]:
    print(seq)

产生50个长度不一（最短3，最长8）的sequences, 其中前十个是:
[5, 3, 9, 9, 5]
[3, 6, 7, 6]
[4, 3, 5, 7, 6, 3]
[6, 3, 4, 3, 6]
[2, 3, 9, 3, 3]
[4, 8, 2, 4, 9, 7, 8, 7]
[6, 9, 2, 7, 3, 3]
[9, 2, 7]
[2, 2, 5, 2, 5, 2]
[5, 4, 2, 7, 8, 5]


# 1.使用seq2seq库实现seq2seq模型（encoder 部分不变）

In [4]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
mode = tf.contrib.learn.ModeKeys.TRAIN

with tf.name_scope('minibatch'):
    encoder_inputs = tf.placeholder(tf.int32, [None, None], name='encoder_inputs')
    
    encoder_inputs_length = tf.placeholder(tf.int32, [None], name='encoder_inputs_length')
    
    decoder_targets = tf.placeholder(tf.int32, [None, None], name='decoder_targets')
    
    decoder_inputs = tf.placeholder(shape=(None, None),dtype=tf.int32,name='decoder_inputs')
    
    #decoder_inputs_length和decoder_targets_length是一样的
    decoder_inputs_length = tf.placeholder(shape=(None,),
                                            dtype=tf.int32,
                                            name='decoder_inputs_length')
    

    
def _create_rnn_cell():
    def single_rnn_cell(encoder_hidden_units):
        # 创建单个cell，这里需要注意的是一定要使用一个single_rnn_cell的函数，不然直接把cell放在MultiRNNCell
        # 的列表中最终模型会发生错误
        single_cell = rnn.LSTMCell(encoder_hidden_units)
        #添加dropout
        single_cell = rnn.DropoutWrapper(single_cell, output_keep_prob=0.5)
        return single_cell
            #列表中每个元素都是调用single_rnn_cell函数
            #cell = rnn.MultiRNNCell([single_rnn_cell() for _ in range(self.num_layers)])
    cell = rnn.MultiRNNCell([single_rnn_cell(encoder_hidden_units) for _ in range(1)])
    return cell 


## 2.Candidate Sampling实现                          
### 实际过程中vocab_size过大，在计算loss生产one_hot的时候传统的softmax由于要计算每一个类的logits就会有问题
### 具体参照这篇论文https://arxiv.org/pdf/1409.0473v7.pdf

In [5]:
num_samples = 5

w_sample = tf.get_variable('proj_w', [vocab_size,encoder_hidden_units])
#w_t = tf.transpose(w)
b_sample = tf.get_variable('proj_b', [vocab_size])

    # 调用sampled_softmax_loss函数计算sample loss，这样可以节省计算时间
def sample_loss(logits, labels):
    labels = tf.cast(labels, tf.int64)
    labels = tf.reshape(labels, [-1, 1])
    logits = tf.cast(logits, tf.float32)
    #logits = tf.reshape(labels, [-1, 1])
    #decoder_logits_train = tf.unstack(logits,axis=1)
    #decoder_targets = tf.unstack(labels,axis=1)
    return tf.cast(tf.nn.sampled_softmax_loss(w_sample, b_sample, labels=labels, inputs=logits,
                                      num_sampled=num_samples, num_classes=vocab_size),tf.float32)
softmax_loss_function = sample_loss


## tensorflow seq2seq.sequence_loss接口：
![alt text](figure/seq2seq_loss接口.png)

## tensorflow tf.nn.sampled_softmax_loss接口：
![alt text](figure/sampled_softmax_loss接口.png)

## 1.定义encoder部分

In [6]:
with tf.variable_scope('encoder'):
    # 创建LSTMCell
    encoder_cell = _create_rnn_cell()
    # 构建embedding矩阵,encoder和decoder公用该词向量矩阵
    embedding = tf.get_variable('embedding', [vocab_size,input_embedding_size])
    encoder_inputs_embedded = tf.nn.embedding_lookup(embedding,encoder_inputs)
    # 使用dynamic_rnn构建LSTM模型，将输入编码成隐层向量。
    # encoder_outputs用于attention，batch_size*encoder_inputs_length*rnn_size,
    # encoder_state用于decoder的初始化状态，batch_size*rnn_szie
    encoder_outputs, encoder_state = tf.nn.dynamic_rnn(encoder_cell, encoder_inputs_embedded,
                                                       sequence_length=encoder_inputs_length,
                                                       dtype=tf.float32)

这里我们使用双向 `dynamic_rnn`:

![alt text](figure/RNN-bidirectional.png)
图片来自于[Colah的blog](http://colah.github.io/posts/2015-09-NN-Types-FP/)

### 2.定义decoder 部分(暂时不添加attention部分)
### 此处不添加output_layer，在sample_loss那里有一层

In [7]:
with tf.variable_scope('decoder'):
    decoder_cell = _create_rnn_cell()
    
    #定义decoder的初始状态
    decoder_initial_state = encoder_state
    
    #定义output_layer
    #output_layer = tf.layers.Dense(encoder_hidden_units,kernel_initializer=tf.truncated_normal_initializer(mean=0.0, stddev=0.1))
    
    decoder_inputs_embedded = tf.nn.embedding_lookup(embedding, decoder_inputs)
    
    # 训练阶段，使用TrainingHelper+BasicDecoder的组合，这一般是固定的，当然也可以自己定义Helper类，实现自己的功能
    training_helper = seq2seq.TrainingHelper(inputs=decoder_inputs_embedded,
                                                        sequence_length=decoder_inputs_length,
                                                        time_major=False, name='training_helper')
    training_decoder = seq2seq.BasicDecoder(cell=decoder_cell, helper=training_helper,
                                                       initial_state=decoder_initial_state,
                                                       output_layer=None)
    
    # 调用dynamic_decode进行解码，decoder_outputs是一个namedtuple，里面包含两项(rnn_outputs, sample_id)
    # rnn_output: [batch_size, decoder_targets_length, vocab_size]，保存decode每个时刻每个单词的概率，可以用来计算loss
    # sample_id: [batch_size], tf.int32，保存最终的编码结果。可以表示最后的答案
    max_target_sequence_length = tf.reduce_max(decoder_inputs_length, name='max_target_len')
    decoder_outputs, _, _ = seq2seq.dynamic_decode(decoder=training_decoder,
                                                          impute_finished=True,
                                                          maximum_iterations=max_target_sequence_length)
    
    #创建一个与decoder_outputs.rnn_output一样的tensor给decoder_logits_train
    decoder_logits_train = tf.identity(decoder_outputs.rnn_output)
    sample_id = decoder_outputs.sample_id
    
    # 根据目标序列长度，选出其中最大值，然后使用该值构建序列长度的mask标志。用一个sequence_mask的例子来说明起作用
    #  tf.sequence_mask([1, 3, 2], 5)
    #  [[True, False, False, False, False],
    #  [True, True, True, False, False],
    #  [True, True, False, False, False]]
    max_target_sequence_length = tf.reduce_max(decoder_inputs_length, name='max_target_len')
    mask = tf.sequence_mask(decoder_inputs_length,max_target_sequence_length, dtype=tf.float32, name='masks')
    print('\t%s' % repr(decoder_logits_train))
    print('\t%s' % repr(decoder_targets))
    print('\t%s' % repr(sample_id))
    loss = seq2seq.sequence_loss(logits=decoder_logits_train,targets=decoder_targets, 
                                weights=mask,softmax_loss_function=softmax_loss_function)

	<tf.Tensor 'decoder/Identity:0' shape=(?, ?, 25) dtype=float32>
	<tf.Tensor 'minibatch/decoder_targets:0' shape=(?, ?) dtype=int32>
	<tf.Tensor 'decoder/decoder/transpose_1:0' shape=(?, ?) dtype=int32>


In [8]:
train_op = tf.train.AdamOptimizer(learning_rate = 0.001).minimize(loss)
sess.run(tf.global_variables_initializer())
def next_feed():
    batch = next(batches)
    
    encoder_inputs_, encoder_inputs_length_ = data_helpers.batch(batch)
    decoder_targets_, decoder_targets_length_ = data_helpers.batch(
        [(sequence) + [EOS] for sequence in batch]
    )
    decoder_inputs_, decoder_inputs_length_ = data_helpers.batch(
        [[EOS] + (sequence) for sequence in batch]
    )
    
    # 在feedDict里面，key可以是一个Tensor
    return {
        encoder_inputs: encoder_inputs_.T,
        decoder_inputs: decoder_inputs_.T,
        decoder_targets: decoder_targets_.T,
        encoder_inputs_length: encoder_inputs_length_,
        decoder_inputs_length: decoder_inputs_length_
    }
x = next_feed()
print('encoder_inputs:')
print(x[encoder_inputs][0,:])
print('encoder_inputs_length:')
print(x[encoder_inputs_length][0])
print('decoder_inputs:')
print(x[decoder_inputs][0,:])
print('decoder_inputs_length:')
print(x[decoder_inputs_length][0])
print('decoder_targets:')
print(x[decoder_targets][0,:])

encoder_inputs:
[6 9 7 7 3 7 0 0]
encoder_inputs_length:
6
decoder_inputs:
[1 6 9 7 7 3 7 0 0]
decoder_inputs_length:
7
decoder_targets:
[6 9 7 7 3 7 1 0 0]


In [9]:
loss_track = []
max_batches = 3001
batches_in_epoch = 100

try:
    # 一个epoch的learning
    for batch in range(max_batches):
        fd = next_feed()
        _, l = sess.run([train_op, loss], fd)
        loss_track.append(l)
        
        if batch == 0 or batch % batches_in_epoch == 0:
            print('batch {}'.format(batch))
            print('  minibatch loss: {}'.format(sess.run(loss, fd)))
            predict_ = sess.run(decoder_outputs.sample_id, fd)
            for i, (inp, pred) in enumerate(zip(fd[encoder_inputs], predict_)):
                print('  sample {}:'.format(i + 1))
                print('    input     > {}'.format(inp))
                print('    predicted > {}'.format(pred))
                if i >= 2:
                    break
            print()
        
except KeyboardInterrupt:
    print('training interrupted')

batch 0
  minibatch loss: 1.7552857398986816
  sample 1:
    input     > [4 3 2 0 0 0 0 0]
    predicted > [ 8 21  3 14  0  0  0  0  0]
  sample 2:
    input     > [2 8 8 2 4 0 0 0]
    predicted > [21 21 21  3  4  3  0  0  0]
  sample 3:
    input     > [8 5 4 4 4 0 0 0]
    predicted > [24 21 24 15 21 18  0  0  0]

batch 100
  minibatch loss: 1.4532331228256226
  sample 1:
    input     > [4 3 2 5 9 4 8 0]
    predicted > [ 0 12 13 13 12 13 13 11  0]
  sample 2:
    input     > [5 8 7 5 4 0 0 0]
    predicted > [24 24 21 18 13 13  0  0  0]
  sample 3:
    input     > [2 4 2 5 9 4 8 8]
    predicted > [24 12  0 13 13 12 13 11 13]

batch 200
  minibatch loss: 1.1243680715560913
  sample 1:
    input     > [9 3 9 2 5 4 0 0]
    predicted > [ 3 12  3 13 21 13 21  0  0]
  sample 2:
    input     > [9 8 5 2 5 4 0 0]
    predicted > [15 13 21  7 13 16 11  0  0]
  sample 3:
    input     > [3 8 5 3 8 2 6 5]
    predicted > [12 12 15  3  7  3  7 18 18]

batch 300
  minibatch loss: 1.181188821